In [ ]:
import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.0'
spark_version='spark-3.1.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:14 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:15 http://ppa.launchpad.net/graph

In [ ]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-06-22 04:48:03--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.1’

postgresql-42.2.16. 100%[===================>] 979.38K  4.84MB/s    in 0.2s    

2021-06-22 04:48:04 (4.84 MB/s) - ‘postgresql-42.2.16.jar.1’ saved [1002883/1002883]



In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-Challenge-furniture-reviews").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

### Load Amazon Data into Spark DataFrame

In [ ]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Furniture_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Furniture_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   24509695|R3VR960AHLFKDV|B004HB5E0E|     488241329|Shoal Creek Compu...|       Furniture|          4|            0|          0|   N|                Y|... desk is very ...|This desk is very...| 2015-08-31|
|         US|   34731776|R16LGVMFKIUT0G|B0042TNMMS|     205864445|Dorel Home Produc...|       Furniture|          5|    

### Create Vine DataFrame for Analysis for Furniture Data Set

In [ ]:
# Create the vine dataFrame
vine_furniture_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_furniture_df.show(5)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3VR960AHLFKDV|          4|            0|          0|   N|                Y|
|R16LGVMFKIUT0G|          5|            0|          0|   N|                Y|
|R1AIMEEPYHMOE4|          5|            1|          1|   N|                Y|
|R1892CCSZWZ9SR|          3|            0|          0|   N|                Y|
|R285P679YWVKD1|          3|            0|          0|   N|                N|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



### Transform Vine DataFrame

In [ ]:
# Filter the data and create a new DataFrame to retrieve all the rows where total_votes 
# count is equal to or greater than 20 to pick reviews that are more likely to be helpful 
# and to avoid having division by zero errors later on.
reviews20_furniture_df = vine_furniture_df.filter("total_votes>=20")
reviews20_furniture_df.show(5)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RL8D0KJ0J9L0O|          5|          152|        165|   N|                Y|
|R1BEINAIQFBRJC|          5|           21|         23|   N|                Y|
|R2L59KIJH302P9|          4|           26|         26|   N|                Y|
| RR99CPG695T0I|          5|          215|        248|   N|                N|
|R1XQNKKUPCMWVO|          5|           43|         44|   N|                Y|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



In [ ]:
reviews20_furniture_df.count()

18739

In [ ]:
# Filter reviews20_furniture_df created above to create a new DataFrame to retrieve all the rows
# where the number of helpful_votes divided by total_votes is equal to or greater than 50%.

helpful_furniture_df = reviews20_furniture_df.filter((reviews20_furniture_df["helpful_votes"]/reviews20_furniture_df["total_votes"]) > 0.5)
helpful_furniture_df.show(5)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RL8D0KJ0J9L0O|          5|          152|        165|   N|                Y|
|R1BEINAIQFBRJC|          5|           21|         23|   N|                Y|
|R2L59KIJH302P9|          4|           26|         26|   N|                Y|
| RR99CPG695T0I|          5|          215|        248|   N|                N|
|R1XQNKKUPCMWVO|          5|           43|         44|   N|                Y|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



In [ ]:
helpful_furniture_df.count()

18135

### Filter Transformed Vine DataFrame for Reviews for Vine Program ("Paid")

In [ ]:
# Filter helpful_furniture_df created in Step 2 to create a new DataFrame to retrieve all the rows
# where a review was written as part of the Vine program (paid), vine == 'Y"
vine_furniture_Y_df = helpful_furniture_df.filter(helpful_furniture_df["vine"] == "Y")
vine_furniture_Y_df.show(5)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2BQOD1R0228FN|          3|           17|         26|   Y|                N|
| RC31RUPFOHBHQ|          4|          102|        117|   Y|                N|
| REN3N1WITLF1Y|          5|           33|         37|   Y|                N|
| R71RZQ9UZVG47|          4|           38|         47|   Y|                N|
|R38NMQBH88HLM6|          4|           18|         24|   Y|                N|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



In [ ]:
# Total number of vine reviews
vine_furniture_Y_df.count()

136

In [ ]:
# Total number of 5 star vine reviews
# Filter vine reviews for 5 star reviews
vine_furniture_Y_df.filter(vine_furniture_Y_df["star_rating"] == "5").count()

74

In [ ]:
# Percentage of 5 star reviews of the total
round((vine_furniture_Y_df.filter(vine_furniture_Y_df["star_rating"] == "5").count()/vine_furniture_Y_df.count())*100)

54

### Filter Transformed Vine DataFrame for Reviews that are not in Vine Program ("Unpaid")

In [ ]:
# Filter helpful_furniture_df created in Step 2 to create a new DataFrame to retrieve all the rows
# where a review was not part of the Vine program (paid), vine == 'N"
vine_furniture_N_df = helpful_furniture_df.filter(helpful_furniture_df["vine"] == "N")
vine_furniture_N_df.show(5)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RL8D0KJ0J9L0O|          5|          152|        165|   N|                Y|
|R1BEINAIQFBRJC|          5|           21|         23|   N|                Y|
|R2L59KIJH302P9|          4|           26|         26|   N|                Y|
| RR99CPG695T0I|          5|          215|        248|   N|                N|
|R1XQNKKUPCMWVO|          5|           43|         44|   N|                Y|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



In [ ]:
# Total number of reviews that are not vine program ("Unpaid")
vine_furniture_N_df.count()

17999

In [ ]:
# Total number of 5 star unpaid reviews
# Filter unpaid reviews for 5 star reviews
vine_furniture_N_df.filter(vine_furniture_N_df["star_rating"] == "5").count()

8480

In [ ]:
# Percentage of 5 star reviews of the total
round((vine_furniture_N_df.filter(vine_furniture_N_df["star_rating"] == "5").count()/vine_furniture_N_df.count())*100)

47

In [ ]:
# filter helpful_furniture_df to include only reviews where purchases were verified
verified_furniture_df = helpful_furniture_df.filter(helpful_furniture_df["verified_purchase"] == "Y")
verified_furniture_df.show(5)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RL8D0KJ0J9L0O|          5|          152|        165|   N|                Y|
|R1BEINAIQFBRJC|          5|           21|         23|   N|                Y|
|R2L59KIJH302P9|          4|           26|         26|   N|                Y|
|R1XQNKKUPCMWVO|          5|           43|         44|   N|                Y|
|R3JUXVCT1NSK2A|          3|           25|         26|   N|                Y|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



In [ ]:
verified_furniture_df.count()

15073

In [ ]:
# Filter verified_furniture_df created above to create a new DataFrame to retrieve all the rows
# where a review was written as part of the Vine program (paid), vine == 'Y"
verified_furniture_Y_df = verified_furniture_df.filter(verified_furniture_df["vine"] == "Y")
verified_furniture_Y_df.show(5)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1CZV9N2YLJAP7|          5|          123|        136|   Y|                Y|
|R1MLK6XUMA9CP1|          5|           62|         70|   Y|                Y|
|R3E0BJMY32EOIP|          4|           17|         22|   Y|                Y|
+--------------+-----------+-------------+-----------+----+-----------------+

